In [1]:
import datetime
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
import sys
from datetime import timedelta  

if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO
    
from IPython.display import HTML
def View(df):
    css = """<style>
    table { border-collapse: collapse; border: 3px solid #eee; }
    table tr th:first-child { background-color: #eeeeee; color: #333; font-weight: bold }
    table thead th { background-color: #eee; color: #000; }
    tr, th, td { border: 1px solid #ccc; border-width: 1px 0 0 1px; border-collapse: collapse;
    padding: 3px; font-family: monospace; font-size: 10px }</style>
    """
    s  = '<script type="text/Javascript">'
    s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
    s += 'win.document.body.innerHTML = \'' + (df.to_html() + css).replace("\n",'\\') + '\';'
    s += '</script>'
    return(HTML(s+css))    
    
print("Current Working directory " , os.getcwd())

Current Working directory  C:\Users\User\GitHub\WebET_Analysis\prolific


In [2]:
data_et = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et.csv')
data_et_fix = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_fix.csv')
data_et_choice = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_et_choice.csv')

data_trial = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial.csv')
data_trial_fix = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_fix.csv')
data_trial_choice = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_trial_choice.csv')

data_prolific = pd.read_csv(r'C:/Users/User/GitHub/WebET_Analysis/data_jupyter/data_prolific.csv') \

overview = pd.DataFrame(
    [
        [len(data_et)],
        [len(data_et_fix)],
        [len(data_et_choice)],
        [len(data_trial)],
        [len(data_trial_fix)],
        [len(data_trial_choice)],
        [len(data_prolific)]
    ], 
    columns=['length'],
    index=['data_et', 'data_et_fix', 'data_et_choice', 
           'data_trial', 'data_trial_fix', 'data_trial_choice', 
           'data_prolific'])
print(overview)

                   length
data_et            618566
data_et_fix         57762
data_et_choice     143127
data_trial          30362
data_trial_fix        918
data_trial_choice    4423
data_prolific          94


# Approve subjects

In [3]:
if 'trial_length' in data_prolific.columns: 
    print('trial_length already added!')
else:
    output = []
    for subject in data_trial['run_id'].unique():
        prolificID = data_trial.loc[data_trial['run_id']==subject, 'prolificID'].unique()[0]
        trial_length = len(data_trial.loc[data_trial['run_id']==subject, 'trial_index'].unique())
        output.append([subject, prolificID, trial_length])
    output = pd.DataFrame(output,
                 columns=['run_id', 'prolificID', 'trial_length']
                )

data_prolific = data_prolific.merge(
    output, 
    on=['run_id', 'prolificID'], 
    how='left')

In [4]:
data_et = data_et.merge(
    data_trial.loc[:, ['run_id', 'chinFirst', 'trial_index', 'trial_type', 'task_nr']], 
    on=['run_id', 'trial_index'], 
    how='left'
)

In [5]:
output = []
for subject in data_et['run_id'].unique():
    m_x_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].mean()
    m_x_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].mean()

    m_y_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'y'
    ].mean()
    m_y_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'y'
    ].mean()

    m_count_fix = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-fix-object') &
        ~((data_et['chinFirst']==1) & (data_et['task_nr']==2) |
          (data_et['chinFirst']==0) & (data_et['task_nr']==3) 
         ), 
        'x'
    ].count()

    m_count_choice = data_et.loc[
        (data_et['run_id']==subject) &
        (data_et['trial_type']=='eyetracking-choice'), 
        'x'
    ].count()
    
    m_choseTop = data_trial_choice.loc[data_trial_choice['run_id']==subject,
                                       'choseTop'].mean()

    output.append([subject, m_x_fix, m_y_fix, m_x_choice, m_y_choice, 
                   m_count_fix, m_count_choice, 
                   m_choseTop])

output = pd.DataFrame(output, 
                      columns=['run_id', 'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 
                               'm_count_fix', 'm_count_choice',
                               'm_choseTop'])
output

data_prolific = data_prolific.merge(output, on='run_id', how='left')

# These subjects await review

In [6]:
data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'trial_length',
     'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
     'm_choseTop']
].sort_values(by='prolificID')

,run_id,prolificID,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,m_choseTop


In [7]:
data_prolific.loc[
    data_prolific['prolificID'].isin(
    ['5fea6632bf9ae4a79153efdf', 
         '5b8969006651ea000118e42e', 
         '5fb2af792942a58ffe303948', 
         '5edc20443467e28ec4e30f93',
         '5fa1192cf99e161a5cfad1cd', 
         '5d430fdf871f1700017b2a81'
        ]
    ), 
    ['run_id', 'prolificID', 'trial_length',
     'm_x_fix', 'm_y_fix', 'm_x_choice', 'm_y_choice', 'm_count_fix', 'm_count_choice',
     'm_choseTop']
].sort_values(by='prolificID')

,run_id,prolificID,trial_length,m_x_fix,m_y_fix,m_x_choice,m_y_choice,m_count_fix,m_count_choice,m_choseTop
15,12.0,5b8969006651ea000118e42e,518.0,0.350426,0.382455,0.353909,0.366253,2073.0,2616.0,0.462500
71,70.0,5d430fdf871f1700017b2a81,407.0,0.265550,0.464537,1.944483,0.170135,61.0,285.0,NaN
86,NaN,5edc20443467e28ec4e30f93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
49,48.0,5fa1192cf99e161a5cfad1cd,520.0,0.516436,0.451000,0.521618,0.499869,2923.0,5158.0,0.544304
66,61.0,5fb2af792942a58ffe303948,271.0,0.517327,0.508298,0.535167,0.454149,967.0,1247.0,0.595238
11,9.0,5fea6632bf9ae4a79153efdf,518.0,0.462945,0.394411,0.531769,0.309021,212.0,478.0,NaN


In [8]:
if 'choice_rt' in data_prolific.columns:
    data_prolific = data_prolific.drop(columns='choice_rt')
grouped = data_trial \
    .loc[
        data_trial['trial_type']=='eyetracking-choice', 
        :] \
    .groupby(['prolificID'])['trial_duration_exact'].mean() \
    .reset_index() \
    .rename(columns={'trial_duration_exact': 'choice_rt'})

data_prolific = data_prolific.merge(
    grouped, 
    on='prolificID', 
    how='left')

data_prolific.loc[
    data_prolific['status']=='AWAITING REVIEW', 
    ['run_id', 'prolificID', 'webcam_fps', 
     'choice_rt', 'bonus_USD', 'bonus_delay']
] \
    .sort_values(by='run_id')

,run_id,prolificID,webcam_fps,choice_rt,bonus_USD,bonus_delay


# data_pay

In [9]:
data_pay = data_prolific.loc[
    data_prolific['status']=='APPROVED', 
    [
        'run_id', 'prolificID', 
        'age',  'Country of Birth', 'Current Country of Residence', 'First Language',
        'Nationality', 'Sex',
        'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
        'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
        'bonus_USD', 'bonus_delay'
    ]
]
print(len(data_pay))

50


# Bonus payment

Reformat payments

In [10]:
data_pay['bonus_delay'] = data_pay['bonus_delay'].astype(str)
data_pay['bonus_delay'] = data_pay['bonus_delay'] \
    .replace(['Today', 'Tomorrow', '7 days', 
          '15 days', '30 days', '90 days', 
          '180 days'], 
         [0, 1, 7, 15, 30, 90, 180]) \
    .astype(float)

data_pay['bonus_USD'] = data_pay['bonus_USD'].astype(str)
data_pay['bonus_USD'] = data_pay['bonus_USD'] \
    .replace({'\$':''}, regex = True) \
    .replace('50 cent', 0.5) \
    .astype(float)

data_pay.loc[:, ['bonus_USD', 'bonus_delay']].head(5)

,bonus_USD,bonus_delay
0,4.5,7.0
3,4.5,30.0
4,4.5,30.0
5,4.0,30.0
10,5.0,30.0


## Replace empty values with the best bonus reward

In [11]:
# For two subjects, the bonus payment was not recorded
data_pay.loc[
    data_pay['prolificID'].isin(
        ['5fea6632bf9ae4a79153efdf', 
         '5b8969006651ea000118e42e', 
         '5fb2af792942a58ffe303948', 
         '5edc20443467e28ec4e30f93',
         '5fa1192cf99e161a5cfad1cd', 
         '5d430fdf871f1700017b2a81'
        ]), ['bonus_USD', 'bonus_delay']] = [5, 1]

data_pay.loc[data_pay['prolificID'].isin(
        ['5fea6632bf9ae4a79153efdf', 
         '5b8969006651ea000118e42e', 
         '5fb2af792942a58ffe303948', 
         '5edc20443467e28ec4e30f93', 
         '5d430fdf871f1700017b2a81'
        ]), ['run_id', 'prolificID', 'bonus_USD', 'bonus_delay']]

,run_id,prolificID,bonus_USD,bonus_delay
11,9.0,5fea6632bf9ae4a79153efdf,5.0,1.0
15,12.0,5b8969006651ea000118e42e,5.0,1.0
66,61.0,5fb2af792942a58ffe303948,5.0,1.0
71,70.0,5d430fdf871f1700017b2a81,5.0,1.0
86,NaN,5edc20443467e28ec4e30f93,5.0,1.0


In [12]:
data_pay.loc[data_pay[['bonus_USD', 'bonus_delay']].isnull().any(axis=1), :]

,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,Country of Birth,entered_code,session_id,started_datetime,completed_date_time,time_taken,bonus_USD,bonus_delay


## Bonus in other currencies

In [13]:
data_pay['bonus_GBP'] = data_pay['bonus_USD'] * 0.75
data_pay['bonus_EUR'] = data_pay['bonus_GBP'] * 1.13

## When is the bonus due?

In [14]:
def add_completed_date(data, data_trial):
    output = []

    for subject in data_trial['run_id'].unique():
        thisSubject = data_trial.loc[data_trial['run_id']==subject] \
            .reset_index(drop=True)
        date_time_obj = datetime.datetime.strptime(
            thisSubject.loc[0, 'recorded_at'], '%Y-%m-%d %H:%M:%S')

        output.append([thisSubject.loc[0, 'run_id'], date_time_obj.date()])
        
    output = pd.DataFrame(output, columns=['run_id', 'date']) 
    
    if 'date' in data.columns: data = data.drop(columns=['date'])
    data = data.merge(output, on='run_id', how='left')
    return data

data_pay = add_completed_date(data_pay, data_trial)
    
data_pay['completed_date']=data_pay.apply(
    lambda x: datetime.datetime.strptime(
        x['completed_date_time'], '%Y-%m-%d %H:%M:%S.%f') \
            .date(),
    axis=1)

data_pay['due_on'] = data_pay['completed_date'] + \
    data_pay['bonus_delay'].map(datetime.timedelta) 

print(len(data_pay))
data_pay.loc[:, 
               [
                   'prolificID', 'run_id',
                   'Nationality', 'Sex',  
                   'bonus_USD', 'bonus_GBP', 'bonus_EUR', 
                   'completed_date', 'bonus_delay', 'due_on'
               ]
              ]. sort_values(by='due_on')


50


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\arrays\datetimelike.py:1342: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
  warnings.warn(


,prolificID,run_id,Nationality,Sex,bonus_USD,bonus_GBP,bonus_EUR,completed_date,bonus_delay,due_on
18,5f0cb319d29147695796a208,37.0,Italy,Male,3.0,2.250,2.54250,2021-01-18,0.0,2021-01-18
17,5f8472685956c40c720d0936,36.0,Italy,Male,0.5,0.375,0.42375,2021-01-18,0.0,2021-01-18
7,5c5684ef9d244c0001b29f1e,11.0,United States,Male,3.5,2.625,2.96625,2021-01-18,0.0,2021-01-18
14,5f561a95aa1c4ea13672f138,28.0,Italy,Male,2.5,1.875,2.11875,2021-01-18,0.0,2021-01-18
9,5f77a902fc647a327b77004a,24.0,United Kingdom,Female,3.0,2.250,2.54250,2021-01-18,0.0,2021-01-18
13,5eeaa0ffaa6af11cf32ce057,30.0,Poland,Male,3.0,2.250,2.54250,2021-01-18,0.0,2021-01-18
49,5fea19869bf07d97b4a87ae8,130.0,Spain,Male,4.0,3.000,3.39000,2021-01-19,0.0,2021-01-19
21,5f4756db83588013e9e31bfc,93.0,South Africa,Male,2.0,1.500,1.69500,2021-01-19,0.0,2021-01-19
28,5cf67b30c5cf0e00010c00fc,90.0,Spain,Female,3.0,2.250,2.54250,2021-01-19,0.0,2021-01-19
30,5c6cb4163023800001e71898,81.0,Greece,Male,3.5,2.625,2.96625,2021-01-19,0.0,2021-01-19


## What bonus is due today? 

In [15]:
bonus_due_today = data_pay.loc[data_pay['due_on']==datetime.datetime.now().date(), :]
bonus_due_today['bonus_GBP'] = bonus_due_today['bonus_GBP'].round(2)
bonus_due_today

,run_id,prolificID,age,Country of Birth,Current Country of Residence,First Language,Nationality,Sex,status,reviewed_at_datetime,...,started_datetime,completed_date_time,time_taken,bonus_USD,bonus_delay,bonus_GBP,bonus_EUR,date,completed_date,due_on


In [16]:
bonus_due_today.loc[: , ['prolificID', 'bonus_GBP']] \
    .to_csv('C:/Users/User/GitHub/WebET_Analysis/prolific/bonusToday.csv', 
            index=False, 
            header=False
           )
print(bonus_due_today['due_on'].unique())

[]


## Bonus left to pay

In [28]:
data_pay.columns

Index(['run_id', 'prolificID', 'age', 'Country of Birth',
       'Current Country of Residence', 'First Language', 'Nationality', 'Sex',
       'status', 'reviewed_at_datetime', 'Country of Birth', 'entered_code',
       'session_id', 'started_datetime', 'completed_date_time', 'time_taken',
       'bonus_USD', 'bonus_delay', 'bonus_GBP', 'bonus_EUR', 'date',
       'completed_date', 'due_on', 'basic_GBP'],
      dtype='object')

In [29]:
data_pay.loc[
    data_pay['due_on']>datetime.datetime.now().date(), 
    ['prolificID', 'completed_date', 'bonus_GBP', 'due_on']]

,prolificID,completed_date,bonus_GBP,due_on
1,5f96fc2552ce3f040af63bbc,2021-01-18,3.375,2021-02-17
2,5ea3f75481a44e39aa318256,2021-01-18,3.375,2021-02-17
3,5fd41bf508717e1833d8aed7,2021-01-18,3.000,2021-02-17
4,5b4e65de09f5af0001080f45,2021-01-18,3.750,2021-02-17
11,5cf0ccf488abf10019fb775d,2021-01-18,3.375,2021-04-18
15,5ea94df3a879f002f3e9a812,2021-01-18,3.375,2021-04-18
16,5f881c7d29650a26096e77a3,2021-01-18,3.375,2021-02-17
19,5f946436ae32a127565d9405,2021-01-18,3.000,2021-02-17
20,5f1aac6058391f01dacbf619,2021-01-19,3.375,2021-02-18
24,5f779405dacc842fc44bc767,2021-01-19,3.000,2021-02-18


In [18]:
left_bonus = data_pay.loc[
    data_pay['due_on']>datetime.datetime.now().date(), 
    'bonus_GBP'].sum()

left_bonus * 1.2 + (left_bonus*0.2*0.2)

71.61

# Total costs

## Basic payment

In [19]:
data_pay['basic_GBP'] = 2.25
data_pay.loc[:, ['basic_GBP', 'bonus_GBP']].head(5)

,basic_GBP,bonus_GBP
0,2.25,3.375
1,2.25,3.375
2,2.25,3.375
3,2.25,3.000
4,2.25,3.750


## Total

In [20]:
print('N =' + str(len(data_pay)) + '\n')

reward_basic = data_pay['basic_GBP'].sum()
service_fee = reward_basic * 0.2
vat = reward_basic * 0.2*0.2
total_basic = data_pay['basic_GBP'].sum() * 1.4
total_bonus = data_pay['bonus_GBP'].sum() * 1.4
total = total_basic + total_bonus

overview = pd.DataFrame(
    [
        [reward_basic], 
        [service_fee], [vat], 
        [total_basic], [total_bonus], [total]
    ], 
    columns=['GBP'],
    index= ['reward_basic', 
        'service_fee', 'vat', 
        'total_basic', 'total_bonus', 'total'])
print(str(overview) + '\n')

print('Total in €, incl. transaction fee: ' + str(total * 1.14))

N =50

                 GBP
reward_basic  112.50
service_fee    22.50
vat             4.50
total_basic   157.50
total_bonus   215.25
total         372.75

Total in €, incl. transaction fee: 424.93499999999995


## Total already paid 
Basic + bonus until today. Use that to compare with prolific bills

In [21]:
bonus_GBP_already_paid = data_pay.loc[
    data_pay['due_on']<=datetime.datetime.now().date(), 
    'bonus_GBP'].sum()

reward_basic = data_pay['basic_GBP'].sum()
service_fee = (reward_basic + bonus_GBP_already_paid) * 0.2
vat = (reward_basic + bonus_GBP_already_paid) * 0.2*0.2
total_basic = data_pay['basic_GBP'].sum() * 1.4
total_bonus = bonus_GBP_already_paid * 1.4
total = total_basic + total_bonus

overview = pd.DataFrame(
    [
        [reward_basic], 
        [service_fee], [vat], 
        [total_basic], [total_bonus], [total]
    ], 
    columns=['GBP'],
    index= ['reward_basic', 
        'service_fee', 'vat', 
        'total_basic', 'total_bonus', 'total'])
print(str(overview) + '\n')

print('Total in €, incl. transaction fee: ' + str(total * 1.14))

                 GBP
reward_basic  112.50
service_fee    41.70
vat             8.34
total_basic   157.50
total_bonus   134.40
total         291.90

Total in €, incl. transaction fee: 332.76599999999996


# Prognosis for the larger sample

In [22]:
def cost_prognosis(n):
    reward_basic = n * data_pay['basic_GBP'].mean()
    reward_bonus = n * data_pay['bonus_GBP'].mean()
        
    total_reward = reward_basic + reward_bonus
    service_fee  = total_reward * 0.2
    vat          = total_reward * 0.2*0.2
    total_basic  = reward_basic + reward_basic*0.2 + reward_basic*0.2*0.2
    total_bonus  = reward_bonus + reward_bonus*0.2 + reward_bonus*0.2*0.2
    total        = total_reward + service_fee + vat

    overview = pd.DataFrame(
        [
            [reward_basic], [reward_bonus], [total_reward],
            [service_fee], [vat], 
            [total_basic], [total_bonus], [total], [total * 1.14]
        ], 
        columns=[n],
        index= ['reward_basic', 'reward_bonus', 'total_reward',
            'service_fee', 'vat', 
            'total_basic', 'total_bonus', 'total GBP', 'total EUR'])
    overview = round(overview, 2)
    return(overview)

print('Budget: ' + str(round(2101*0.88, 2)) + ' GBP / ' + 
      '2101.00 EUR / ' +
      'n=' + str(int(np.floor((2101*0.88)/6.6)))
     )

print('Paid:   1700.50 GBP / ' + 
      str(round(1700.5*1.15, 2)) + ' EUR / ' + 
      'n=' + str(int(np.floor(1700.5/6.6))))
buffer_EUR = 2101-1955.57
buffer_GBP = buffer_EUR * 0.88
print('Buffer: ' + str(round(buffer_GBP, 2)) + ' /  ' + 
      str(round(buffer_EUR, 2)) + ' EUR / n=' + 
      str(int(np.floor(buffer_GBP/6.6))) + '\n')
print('Collect 207 more participants.')

pd.concat([
    cost_prognosis(1),
    cost_prognosis(50),
    cost_prognosis(257), 
    cost_prognosis(207)
], axis=1)

Budget: 1848.88 GBP / 2101.00 EUR / n=280
Paid:   1700.50 GBP / 1955.57 EUR / n=257
Buffer: 127.98 /  145.43 EUR / n=19

Collect 207 more participants.


,1,50,257,207
reward_basic,2.25,112.50,578.25,465.75
reward_bonus,3.08,153.75,790.28,636.53
total_reward,5.32,266.25,1368.52,1102.28
service_fee,1.07,53.25,273.71,220.46
vat,0.21,10.65,54.74,44.09
total_basic,2.79,139.50,717.03,577.53
total_bonus,3.81,190.65,979.94,789.29
total GBP,6.60,330.15,1696.97,1366.82
total EUR,7.53,376.37,1934.55,1558.18


# Export modified prolific data

In [23]:
data_pay.to_csv('C:/Users/User/GitHub/WebET_Analysis/prolific/data_pay.csv', index=False, header=True)

# Feedback

In [24]:
print('Success! Script ran through.')

Success! Script ran through.
